<a href="https://colab.research.google.com/github/NinaNikolova/data_mining/blob/main/02_Pandas_Optimizations_Pt_1_Memory_Efficiency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ефективно използване на паметта в Python
#### Python е език с динамично управление на паметта, което улеснява програмистите, но може да доведе до неефективно използване на ресурсите. Ето няколко начина за оптимизация на паметта:
## Използване на вградени структури от данни ефективно - Избягвайте излишни копия на данни. Например, вместо list, използвайте set за бързо търсене или tuple, ако данните не се променят.; Използвайте генератори вместо списъци, когато е възможно - yield(Спестява памет, защото не съхранява всички числа в списък.)
##  Използване на __slots__ в класове: Стандартните Python обекти използват речник (__dict__) за атрибутите, което изисква повече памет.; Ако знаете предварително атрибутите на класа, може да използвате __slots__; Това намалява използваната памет при създаване на много обекти.
## Избягване на ненужни референции и циклични зависимости - Ако често добавяте или премахвате елементи в началото на списък, collections.deque е по-ефективен
## Използване на array и numpy за числови операции - list използва повече памет от array; numpy оптимизира работата с големи числови масиви
## Използване на memoryview за работа с бинарни данни- memoryview позволява достъп до съществуваща памет без копиране
## Работа с големи файлове чрез mmap - mmap позволява работа с големи файлове без зареждане на целия файл в паметта

In [1]:
import pandas as pd
import numpy as np

## Създаване и оптимизация на данни в Pandas

###### Създаване на DataFrame - Кодът представя създаване на случайно генериран Pandas DataFrame и техники за оптимизация на паметта, като кастинг на данни (преобразуване на типове). Функцията get_dataset(size) генерира DataFrame с определен брой редове (size), съдържащ следните колони:

In [2]:
def get_dataset(size):
    df = pd.DataFrame()
    df['position'] = np.random.choice(['left','middle','right'], size)  # Случайна позиция
    df['age'] = np.random.randint(1, 50, size) # Случайна възраст (1-49)
    df['team'] = np.random.choice(['red','blue','yellow','green'], size)  # Отбор
    df['win'] = np.random.choice(['yes','no'], size) # Победа/Загуба
    df['prob'] = np.random.uniform(0, 1, size)  # Случайна вероятност (0-1)
    return df

In [3]:
df = get_dataset(1_000_000) # Създава 1 милион реда
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   position  1000000 non-null  object 
 1   age       1000000 non-null  int64  
 2   team      1000000 non-null  object 
 3   win       1000000 non-null  object 
 4   prob      1000000 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 38.1+ MB


##Ранкиране на стойности в групи
######Тестове за ефективност на rank() с %timeit:

In [4]:
%timeit df['age_rank'] = df.groupby(['team','position'])['age'].rank() #groupby(...).rank() изчислява ранг (позиция) на стойностите в групирани подмножества- Проверка на бързодействието на тези операции.
%timeit df['prob_rank'] = df.groupby(['team','position'])['prob'].rank()
%timeit df['win_prob_rank'] = df.groupby(['team','position','win'])['prob'].rank()

1.34 s ± 434 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
760 ms ± 29.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
789 ms ± 138 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Оптимизация чрез кастинг на данни

In [5]:
df = get_dataset(1_000_000)
df['position'] = df['position'].astype('category') # По подразбиране position и team се съхраняват като object, което заема много памет. Кастингът към category значително намалява използваната памет.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column    Non-Null Count    Dtype   
---  ------    --------------    -----   
 0   position  1000000 non-null  category
 1   age       1000000 non-null  int64   
 2   team      1000000 non-null  object  
 3   win       1000000 non-null  object  
 4   prob      1000000 non-null  float64 
dtypes: category(1), float64(1), int64(1), object(2)
memory usage: 31.5+ MB


In [6]:
df = get_dataset(1_000_000)
df['position'] = df['position'].astype('category')
df['team'] = df['team'].astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column    Non-Null Count    Dtype   
---  ------    --------------    -----   
 0   position  1000000 non-null  category
 1   age       1000000 non-null  int64   
 2   team      1000000 non-null  category
 3   win       1000000 non-null  object  
 4   prob      1000000 non-null  float64 
dtypes: category(2), float64(1), int64(1), object(1)
memory usage: 24.8+ MB


###  Кастинг на цели числа (int)

Различните типове int заемат различно количество памет:

- int8 (-128 до 127)

- int16 (-32,768 до 32,767)

- int64 (-9 квинт. до 9 квинт.)

In [7]:
df['age']

,age
0,20
1,7
2,45
3,48
4,4
...,...
999995,21
999996,42
999997,29
999998,43


In [8]:
df['age'].min(), df['age'].max() # Проверка на обхвата (1-49)

(1, 49)

In [9]:
df['age'] = df['age'].astype('int8') # int8 е достатъчен
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column    Non-Null Count    Dtype   
---  ------    --------------    -----   
 0   position  1000000 non-null  category
 1   age       1000000 non-null  int8    
 2   team      1000000 non-null  category
 3   win       1000000 non-null  object  
 4   prob      1000000 non-null  float64 
dtypes: category(2), float64(1), int8(1), object(1)
memory usage: 18.1+ MB


### Кастинг на числа с плаваща запетая (float)
###### Промяна на типа на prob

In [10]:
df['prob']

,prob
0,0.663765
1,0.491755
2,0.347723
3,0.711990
4,0.399094
...,...
999995,0.203425
999996,0.073364
999997,0.312859
999998,0.398014


In [11]:
df['prob'].astype('float16') #float16 заема по-малко памет, но е по-малко точен

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,prob
0,0.663574
1,0.491699
2,0.347656
3,0.711914
4,0.399170
...,...
999995,0.203369
999996,0.073364
999997,0.312744
999998,0.397949


In [12]:
df['prob'].astype('float32') #float32 е по-добър баланс между точност и използвана памет.

,prob
0,0.663765
1,0.491755
2,0.347723
3,0.711990
4,0.399094
...,...
999995,0.203425
999996,0.073364
999997,0.312859
999998,0.398014


In [13]:
df['prob'] = df['prob'].astype('float32')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column    Non-Null Count    Dtype   
---  ------    --------------    -----   
 0   position  1000000 non-null  category
 1   age       1000000 non-null  int8    
 2   team      1000000 non-null  category
 3   win       1000000 non-null  object  
 4   prob      1000000 non-null  float32 
dtypes: category(2), float32(1), int8(1), object(1)
memory usage: 14.3+ MB


###  Кастинг на булеви (bool) стойности (True/False)
###### Колоната win съдържа "yes" и "no", които се преобразуват в True/False:

In [14]:
df['win']

,win
0,no
1,yes
2,yes
3,yes
4,yes
...,...
999995,no
999996,yes
999997,no
999998,no


In [15]:
df['win'].map({'yes':True, 'no':False}) # map() преобразува текстовите стойности в булеви (bool), което заема само 1 бит вместо цял обект (object).

,win
0,False
1,True
2,True
3,True
4,True
...,...
999995,False
999996,True
999997,False
999998,False


In [16]:
df['win'] = df['win'].map({'yes':True, 'no':False}) # map() преобразува текстовите стойности в булеви (bool), което заема само 1 бит вместо цял обект (object).
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column    Non-Null Count    Dtype   
---  ------    --------------    -----   
 0   position  1000000 non-null  category
 1   age       1000000 non-null  int8    
 2   team      1000000 non-null  category
 3   win       1000000 non-null  bool    
 4   prob      1000000 non-null  float32 
dtypes: bool(1), category(2), float32(1), int8(1)
memory usage: 7.6 MB


## Сравняване преди и слд кастване Casting

In [17]:
def set_dtypes(df):
    df['position'] = df['position'].astype('category')
    df['team'] = df['team'].astype('category')
    df['age'] = df['age'].astype('int8')
    df['prob'] = df['prob'].astype('float32')
    df['win'] = df['win'].map({'yes':True, 'no':False})
    return df

In [18]:
df = get_dataset(1_000_000)
%timeit df['age_rank'] = df.groupby(['team','position'])['age'].rank()
%timeit df['prob_rank'] = df.groupby(['team','position'])['prob'].rank()
%timeit df['win_prob_rank'] = df.groupby(['team','position','win'])['prob'].rank()

490 ms ± 222 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
676 ms ± 304 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
557 ms ± 3.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
df = get_dataset(1_000_000)
df = set_dtypes(df)
%timeit df['age_rank'] = df.groupby(['team','position'])['age'].rank()
%timeit df['prob_rank'] = df.groupby(['team','position'])['prob'].rank()
%timeit df['win_prob_rank'] = df.groupby(['team','position','win'])['prob'].rank() # Булеви стойности (bool) заемат по-малко памет от object тип.

<magic-timeit>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
<magic-timeit>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.


157 ms ± 10.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


<magic-timeit>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.


402 ms ± 52.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<magic-timeit>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.


337 ms ± 8.39 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Larger Data

In [20]:
df = get_dataset(10_000_000)
%timeit df['age_rank'] = df.groupby(['team','position'])['age'].rank()
%timeit df['prob_rank'] = df.groupby(['team','position'])['prob'].rank()
%timeit df['win_prob_rank'] = df.groupby(['team','position','win'])['prob'].rank()

5.54 s ± 279 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
7.43 s ± 1.04 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
8.12 s ± 255 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
df = get_dataset(10_000_000)
df = set_dtypes(df)
%timeit df['age_rank'] = df.groupby(['team','position'])['age'].rank()
%timeit df['prob_rank'] = df.groupby(['team','position'])['prob'].rank()
%timeit df['win_prob_rank'] = df.groupby(['team','position','win'])['prob'].rank()

<magic-timeit>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
<magic-timeit>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.


2.95 s ± 283 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<magic-timeit>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.


5.39 s ± 264 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<magic-timeit>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.


5.66 s ± 329 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
